In [20]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Load Credentials

In [17]:
import os
from google.oauth2.service_account import Credentials

CREDENTIAL_PATHS = list(filter(
    lambda f:
    f.endswith('.json') and f.startswith('gcp_cred'),
    os.listdir(os.path.abspath(''))))
if not CREDENTIAL_PATHS:
    raise FileNotFoundError(
        "No Google Cloud credentials found in the root directory.\nInsure credential file starts with 'gcp_cred' and ends with '.json'")

CREDENTIALS = Credentials.from_service_account_file(CREDENTIAL_PATHS[0])

### Define Model

In [18]:
from langchain_google_vertexai import ChatVertexAI

model = ChatVertexAI(
    model="gemini-1.0-pro-vision",
    temperature=0.9, 
    max_tokens=2048,
    timeout=None,
    max_retries=2,
    region="us-central1",
    credentials=CREDENTIALS,
    project=CREDENTIALS.project_id,
)

### Direct Model Invoke

In [23]:
response = model.invoke("What is the color of an apple?")
print(response.usage_metadata)
print(response.content)

{'input_tokens': 8, 'output_tokens': 19, 'total_tokens': 27}
Apples can vary in color, but the most common colors are red, green, and yellow.


### Chroma Data Retrival

assume data is loaded to chroma db to persistant dir `chroma_db`

In [40]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_chroma import Chroma

embeddings = VertexAIEmbeddings(
    credentials=CREDENTIALS,
    project=CREDENTIALS.project_id,
    model_name="text-multilingual-embedding-002",
)
chroma_db = Chroma(
    persist_directory="chroma_db",
    embedding_function=embeddings,
)
retriever = chroma_db.as_retriever()

### Model RAG Text response

chrom db is loaded with hong kong public transport information

In [45]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. You should answer the user's questions based on the contextinformatino given"),
    ("placeholder", "{conversation}")
])


prompt = template.invoke(
    {
        "conversation": [
            ("human", "Context: {context}\n\n{question}"),
        ]
    }
)
# prompt = hub.pull("rlm/rag-prompt")

question = "what route to Tism Sha Tsui are avalable at Amoy Garden?"

rag_chain = (
    [{
        "conversation": [
            ("human", f"Context: {retriever.pipe(format_docs)}\n\n{RunnablePassthrough()}"),
        ]
    }]
    | template
    | model
    | StrOutputParser()
)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'list'>